# Experiment PAMAP2 with mcfly

This experiment finds an optimal model for the PAMAP2 dataset.

## Import required Python modules

In [1]:
import sys
import os
import numpy as np
import pandas as pd
# mcfly
from mcfly import modelgen, find_architecture, storage


Using TensorFlow backend.


In [11]:
trainsize = 1#500
valsize = 1 #500 
nr_models = 10
nr_epochs = 10
subset_size = trainsize

## Load the data

In [2]:
# Define directory where the results, e.g. json file, will be stored
datapath = '/media/sf_VBox_Shared/timeseries/PAMAP2/PAMAP2_Dataset/cleaned_12activities_allvars/'
resultpath = '/media/sf_VBox_Shared/timeseries/PAMAP2/PAMAP2_Dataset/results_test/' 
if not os.path.exists(resultpath):
        os.makedirs(resultpath)

In [3]:
Xs = []
ys = []

ext = '.npy'
for i in range(9):
    Xs.append(np.load(os.path.join(datapath,'X_'+str(i)+ext)))
    ys.append(np.load(os.path.join(datapath, 'y_'+str(i)+ext)))

In [4]:
print(Xs[0].shape, ys[0].shape)

(2155, 512, 52) (2155, 12)


## Generate models

First step is to create a model architecture. As we do not know what architecture is best for our data we will create a set of models to investigate which architecture is most suitable for our data and classification task. You will need to specificy how many models you want to create with argument 'number_of_models', the type of model which can been 'CNN' or 'DeepConvLSTM', and maximum number of layers per modeltype. See for a full overview of the optional arguments the function documentation of modelgen.generate_models

In [16]:
num_classes = ys[0].shape[1]
np.random.seed(123)
models = modelgen.generate_models(Xs[0].shape,
                                  number_of_classes=num_classes,
                                  number_of_models = nr_models)

/home/dafne/anaconda2/envs/mcfly/lib/python3.5/site-packages/mcfly/modelgen.py:160: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(27, (3, 1), kernel_regularizer=<keras.reg..., data_format="channels_first", kernel_initializer="lecun_uniform", padding="same")`
  dim_ordering='th'))
/home/dafne/anaconda2/envs/mcfly/lib/python3.5/site-packages/mcfly/modelgen.py:160: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(93, (3, 1), kernel_regularizer=<keras.reg..., data_format="channels_first", kernel_initializer="lecun_uniform", padding="same")`
  dim_ordering='th'))
/home/dafne/anaconda2/envs/mcfly/lib/python3.5/site-packages/mcfly/modelgen.py:160: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(67, (3, 1), kernel_regularizer=<keras.reg..., data_format="channels_first", kernel_initializer="lecun_uniform", padding="same")`
  dim_ordering='th'))
/home/dafne/anaconda2/envs/mcfly/lib/python3.5/site-packages/mcfly/modelgen.py:160: UserWarnin

In [19]:
for i, (model, params, model_type) in enumerate(models):
    storage.savemodel(model,resultpath,"model_"+str(i))

## Compare models
Now that the model architectures have been generated it is time to compare the models by training them in a subset of the training data and evaluating the models in the validation subset. This will help us to choose the best candidate model. Performance results are stored in a json file.

In [25]:
def split_train_test(X_list, y_list, j):
    X_train = np.concatenate(X_list[0:j]+X_list[j+1:])
    X_test = X_list[j]
    y_train = np.concatenate(y_list[0:j]+y_list[j+1:])
    y_test = y_list[j]
    return X_train, y_train, X_test, y_test

def split_train_small_val(X_list, y_list, j, trainsize=500, valsize=500):
    X = np.concatenate(X_list[0:j]+X_list[j+1:])
    y = np.concatenate(y_list[0:j]+y_list[j+1:])
    rand_ind = np.random.choice(X.shape[0], trainsize+valsize, replace=False)
    X_train = X[rand_ind[:trainsize]]
    y_train = y[rand_ind[:trainsize]]
    X_val = X[rand_ind[trainsize:]]
    y_val = y[rand_ind[trainsize:]]
    return X_train, y_train, X_val, y_val

In [26]:
from keras.optimizers import Adam
from keras.models import model_from_json

def get_fresh_copy(model, lr):
    model_json = model.to_json()
    model_copy = model_from_json(model_json)
    model_copy.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=lr),
                  metrics=['accuracy'])
    #for layer in model_copy.layers:
    #    layer.build(layer.input_shape)
    return model_copy

In [10]:
models = [(get_fresh_copy(model, params['learning_rate']), params, model_type)  for model, params, model_type in models]

In [12]:
import time
t = time.time()
np.random.seed(123)
histories_list, val_accuracies_list, val_losses_list = [], [], []
for j in range(len(Xs)):
    print('fold '+str(j))
    models = [(get_fresh_copy(model, params['learning_rate']), params, model_type)  for model, params, model_type in models]
    X_train, y_train, X_val, y_val = split_train_small_val(Xs, ys, j, trainsize=trainsize, valsize=valsize)
    histories, val_accuracies, val_losses = find_architecture.train_models_on_samples(X_train, y_train,
                                                                           X_val, y_val,
                                                                           models,
                                                                           nr_epochs=nr_epochs,
                                                                           subset_size=subset_size,
                                                                           verbose=True,
                                                                           outputfile=os.path.join(resultpath, 
                                                                                  'experiment'+str(j)+'.json'),
                                                                           early_stopping=True)
    histories_list.append(histories)
    val_accuracies_list.append(val_accuracies)
    val_losses.append(val_losses)
print(time.time()-t)

fold 0
Training model 0 DeepConvLSTM


/home/dafne/anaconda2/envs/mcfly/lib/python3.5/site-packages/keras/models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


KeyboardInterrupt: 

In [6]:
# Read them all back in
import json
model_jsons = []
for j in len(Xs):
    with open(os.path.join(resultpath, 'experiment'+str(j)+'.json'), 'r') as outfile:
        model_jsons.append(json.load(outfile))

In [12]:
val_accuracies = np.array([[mod['val_acc'][-1] for mod in fold] for fold in model_jsons])

In [13]:
val_acc = np.array([np.array([mod['val_acc'][-1] for mod in fold], dtype='float') for fold in model_jsons])
train_acc = np.array([np.array([mod['train_acc'][-1] for mod in fold], dtype='float') for fold in model_jsons])
train_loss = np.array([np.array([mod['train_loss'][-1] for mod in fold], dtype='float') for fold in model_jsons])
val_loss = np.array([np.array([mod['val_loss'][-1] for mod in fold], dtype='float') for fold in model_jsons])

In [14]:
val_accuracies_avg = val_acc.mean(axis=0)
print('val_accuracies_avg:', val_accuracies_avg)

val_accuracies_avg: [ 0.131  0.285  0.196  0.087  0.436  0.137  0.131  0.544  0.278  0.105
  0.121  0.274  0.436]


In [23]:
best_model_index = np.argmax(val_accuracies_avg)
best_model = storage.loadmodel(resultpath, 'model_'+str(best_model_index))

In [28]:
best_params = model_jsons[0][best_model_index]

## Train the best model for real

Now that we have identified the best model architecture out of our random pool of models we can continue by training the model on the full training sample. For the purpose of speeding up the example we only train the full model on the first 1000 values. You will need to replace this by 'datasize = X_train.shape[0]' in a real world example.

In [ ]:
nr_epochs = 2

np.random.seed(123)
histories, test_accuracies_list, models = [], [], []
for j in range(len(Xs)):
    X_train, y_train, X_test, y_test = split_train_test(Xs, ys, j)
    model_copy = get_fresh_copy(best_model, best_params['learning_rate'])
    datasize = X_train.shape[0]
    
    history = model_copy.fit(X_train[:datasize,:,:], y_train[:datasize,:],
              nb_epoch=nr_epochs, validation_data=(X_test, y_test))
    
    histories.append(history)
    test_accuracies_list.append(history.history['val_acc'][-1] )
    models.append(model_copy)

/home/dafne/anaconda2/envs/mcfly/lib/python3.5/site-packages/keras/models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 14663 samples, validate on 2155 samples
Epoch 1/2


In [ ]:
modelname = 'my_bestmodel'

In [ ]:
for i, model in enumerate(models):
    storage.savemodel(model,resultpath,modelname+str(i))

In [ ]:
print('accuracies: ', test_accuracies_list)

In [ ]:
print(np.mean(test_accuracies_list))